In [ ]:
# 📦 Instalar bibliotecas necessárias
!pip install wordcloud pandas matplotlib seaborn openpyxl plotly


In [ ]:
# 📚 Importação de bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import defaultdict, Counter
import os
import plotly.express as px
from datetime import datetime


In [ ]:
# 📁 Carregamento da base de dados
arquivo = '/content/Manifestacoes_Abril2025.xlsx'
df = pd.read_excel(arquivo)


In [ ]:
# 🛠️ Ajuste de colunas caso o cabeçalho esteja na linha 2
if df.columns[0] != 'COD_MANIFESTACAO':
    df.columns = df.iloc[0]
    df = df[1:]
    df.columns.name = None


In [ ]:
# 🏷️ Renomear colunas principais
df = df.rename(columns={
    "COD_MANIFESTACAO": "COD_MANIFESTACAO",
    "MOTIVO": "MOTIVO",
    "DATA_MANIF": "DATA_MANIF",
    "DIA_SEM": "DIA_SEM",
    "DATA_OCOR": "DATA_OCOR",
    "HORA_OCOR": "HORA_OCOR",
    "LOCAL_REF": "LOCAL_REF",
    "DESCRIÇÃO": "DESCRICAO",
    "CLASSIFICACAO_MANIF": "CLASSIFICACAO_MANIF",
    df.columns[-1]: "LINHA"
})


In [ ]:
# ⏱️ Conversão de tipos
df["DATA_MANIF"] = pd.to_datetime(df["DATA_MANIF"], errors='coerce')
df["DATA_OCOR"] = pd.to_datetime(df["DATA_OCOR"], errors='coerce')
df["HORA_OCOR"] = pd.to_numeric(df["HORA_OCOR"], errors='coerce')


In [ ]:
# 📊 Top 15 motivos de manifestação
plt.figure(figsize=(12, 6))
df["MOTIVO"].value_counts().nlargest(15).plot(kind='barh', color='darkred')
plt.title("Top 15 Motivos de Manifestação")
plt.xlabel("Frequência")
plt.gca().invert_yaxis()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# 📅 Distribuição por dia da semana
dias_ordenados = ['segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira', 'sábado', 'domingo']
df["DIA_SEM"] = df["DIA_SEM"].str.lower()
df["DIA_SEM"] = pd.Categorical(df["DIA_SEM"], categories=dias_ordenados, ordered=True)

plt.figure(figsize=(10, 5))
sns.countplot(data=df, x="DIA_SEM", order=dias_ordenados, palette="Reds")
plt.title("Distribuição de Manifestações por Dia da Semana")
plt.ylabel("Quantidade")
plt.xlabel("Dia da Semana")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# 🔥 Mapa de calor por hora e dia
pivot = df.pivot_table(index="HORA_OCOR", columns="DIA_SEM", aggfunc="size", fill_value=0)
plt.figure(figsize=(12, 6))
sns.heatmap(pivot, cmap="YlOrRd", linewidths=0.3, linecolor='gray')
plt.title("Mapa de Calor: Hora do Dia x Dia da Semana")
plt.xlabel("Dia da Semana")
plt.ylabel("Hora")
plt.tight_layout()
plt.show()


In [ ]:
# ☁️ Nuvem de palavras geral (todas as manifestações)
texto = " ".join(df["DESCRICAO"].dropna().astype(str).tolist())
wordcloud = WordCloud(width=1200, height=600, background_color='white', colormap='inferno', max_words=150).generate(texto)
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Nuvem de Palavras - Todas as Manifestações")
plt.show()


In [ ]:
# 🎯 Foco em reclamações - apenas adjetivos (simulados)
df_reclamacoes = df[df["CLASSIFICACAO_MANIF"].str.upper() == "RECLAMAÇÃO"].copy()

adjetivos_exemplo = [
    'lento', 'sujo', 'barulhento', 'quente', 'frio', 'perigoso', 'velho', 'lotado', 'demorado',
    'inseguro', 'desconfortável', 'escuro', 'caro', 'úmido', 'horrível', 'ruim', 'péssimo',
    'fraco', 'frágil', 'instável', 'precário', 'ineficiente', 'insuficiente', 'revoltante',
    'desumano', 'inaceitável', 'desagradável', 'injusto', 'problemático', 'difícil', 'caótico',
    'quebrado', 'falho', 'irregular', 'frequente', 'inconstante', 'baixa', 'curto', 'agressivo',
    'confuso', 'degradante', 'frustrante', 'impróprio', 'inadequado', 'insuportável', 'lamentável',
    'mofado', 'obsoleto', 'pobre', 'saturado', 'sofrido', 'sufocante', 'tenso', 'triste', 'vazio',
    'interminável', 'irresponsável', 'inoperante', 'defeituoso', 'danificado', 'parado', 'pior'
]
adjetivos_set = set([a.lower() for a in adjetivos_exemplo if len(a) >= 3])

adjetivos_linha = defaultdict(list)
for linha in df_reclamacoes["LINHA"].unique():
    descricoes = df_reclamacoes[df_reclamacoes["LINHA"] == linha]["DESCRICAO"].dropna().astype(str).tolist()
    palavras = " ".join(descricoes).lower().split()
    palavras_filtradas = [p for p in palavras if p in adjetivos_set]
    adjetivos_linha[linha] = palavras_filtradas

output_folder = "/content/nuvens_adjetivos"
os.makedirs(output_folder, exist_ok=True)

for linha, lista_adjetivos in adjetivos_linha.items():
    if len(lista_adjetivos) < 5:
        continue
    texto_final = " ".join(lista_adjetivos)
    wordcloud = WordCloud(width=1000, height=500, background_color='white', colormap='inferno', max_words=100).generate(texto_final)
    wordcloud.to_file(os.path.join(output_folder, f"{linha.replace(' ', '_')}.png"))


In [ ]:
# 📊 Ranking completo de adjetivos por linha
ranking_completo = []
for linha, lista_adjetivos in adjetivos_linha.items():
    contagem = Counter(lista_adjetivos)
    for adjetivo, freq in contagem.items():
        ranking_completo.append({
            "Linha": linha,
            "Adjetivo": adjetivo,
            "Frequência": freq
        })

df_ranking = pd.DataFrame(ranking_completo).sort_values(by=["Linha", "Frequência"], ascending=[True, False])
df_ranking.to_excel("/content/adjetivos_por_linha.xlsx", index=False)

print("✅ Tudo pronto! Arquivos gerados:")
print("- 📁 Nuvens salvas em: /content/nuvens_adjetivos")
print("- 📊 Ranking salvo em: /content/adjetivos_por_linha.xlsx")


In [ ]:
# 🎯 Análise de expressões compostas nas reclamações (bigramas)

from collections import defaultdict, Counter
from wordcloud import WordCloud
import os
import re

# Filtrar apenas reclamações
df_reclamacoes = df[df["CLASSIFICACAO_MANIF"].str.upper() == "RECLAMAÇÃO"].copy()

# Função para extrair expressões de 2 palavras (bigramas)
def extrair_bigramas(texto):
    palavras = re.findall(r'\b\w{3,}\b', texto.lower())
    return [f"{palavras[i]} {palavras[i+1]}" for i in range(len(palavras)-1)]

# Criar dicionário com expressões por linha
expressoes_linha = defaultdict(list)

for linha in df_reclamacoes["LINHA"].unique():
    descricoes = df_reclamacoes[df_reclamacoes["LINHA"] == linha]["DESCRICAO"].dropna().astype(str).tolist()
    for desc in descricoes:
        bigramas = extrair_bigramas(desc)
        expressoes_linha[linha].extend(bigramas)

# Criar pasta para salvar as nuvens
output_folder = "/content/nuvens_expressoes"
os.makedirs(output_folder, exist_ok=True)

# Gerar nuvens de palavras para as 30 expressões mais frequentes por linha
for linha, lista_express in expressoes_linha.items():
    if len(lista_express) < 10:
        continue

    contagem = Counter(lista_express).most_common(30)
    texto_final = " ".join([(" ".join([expr]*freq)) for expr, freq in contagem])

    wordcloud = WordCloud(width=1000, height=500, background_color='white', colormap='inferno', max_words=100, collocations=False).generate(texto_final)
    wordcloud.to_file(os.path.join(output_folder, f"{linha.replace(' ', '_')}_expressoes.png"))

print("✅ Nuvens de expressões compostas geradas com sucesso por linha (reclamações)")


# 🔗 Seção Interativa com Dashboards

In [ ]:
# 📦 Instalar bibliotecas interativas
!pip install plotly dash jupyter-dash pandas openpyxl


In [ ]:
# 📚 Importações
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output


In [ ]:
# 📁 Carregamento da base
arquivo = '/content/Manifestacoes_Abril2025.xlsx'
df = pd.read_excel(arquivo)

# Ajustar colunas caso necessário
if df.columns[0] != 'COD_MANIFESTACAO':
    df.columns = df.iloc[0]
    df = df[1:]
    df.columns.name = None

# Renomear colunas
df = df.rename(columns={
    "COD_MANIFESTACAO": "COD_MANIFESTACAO",
    "MOTIVO": "MOTIVO",
    "DATA_MANIF": "DATA_MANIF",
    "DIA_SEM": "DIA_SEM",
    "DATA_OCOR": "DATA_OCOR",
    "HORA_OCOR": "HORA_OCOR",
    "LOCAL_REF": "LOCAL_REF",
    "DESCRIÇÃO": "DESCRICAO",
    "CLASSIFICACAO_MANIF": "CLASSIFICACAO_MANIF",
    df.columns[-1]: "LINHA"
})

# Converter tipos
df["DATA_MANIF"] = pd.to_datetime(df["DATA_MANIF"], errors='coerce')
df["HORA_OCOR"] = pd.to_numeric(df["HORA_OCOR"], errors='coerce')


In [ ]:
# 🚀 Criação do app Dash por classificação de manifestação
def iniciar_dashboard_por_classificacao(classe):
    dados = df[df["CLASSIFICACAO_MANIF"].str.upper() == classe.upper()].copy()

    app = JupyterDash(__name__)
    app.layout = html.Div([
        html.H1(f"Dashboard Interativo - {classe.title()}", style={"textAlign": "center"}),

        dcc.Dropdown(
            id='linha-dropdown',
            options=[{"label": linha, "value": linha} for linha in sorted(dados["LINHA"].dropna().unique())],
            placeholder="Selecione uma linha (opcional)",
            multi=True
        ),

        dcc.DatePickerRange(
            id='data-range',
            min_date_allowed=dados["DATA_MANIF"].min(),
            max_date_allowed=dados["DATA_MANIF"].max(),
            start_date=dados["DATA_MANIF"].min(),
            end_date=dados["DATA_MANIF"].max()
        ),

        dcc.Graph(id='grafico-motivos'),
        dcc.Graph(id='grafico-horas'),
        dcc.Graph(id='grafico-local')
    ])

    @app.callback(
        Output('grafico-motivos', 'figure'),
        Output('grafico-horas', 'figure'),
        Output('grafico-local', 'figure'),
        Input('linha-dropdown', 'value'),
        Input('data-range', 'start_date'),
        Input('data-range', 'end_date')
    )
    def atualizar_graficos(linhas_selecionadas, data_inicio, data_fim):
        dff = dados.copy()
        if linhas_selecionadas:
            dff = dff[dff["LINHA"].isin(linhas_selecionadas)]
        dff = dff[(dff["DATA_MANIF"] >= data_inicio) & (dff["DATA_MANIF"] <= data_fim)]

        fig1 = px.bar(dff["MOTIVO"].value_counts().nlargest(10).reset_index(),
                      x="index", y="MOTIVO", title="Top 10 Motivos")
        fig1.update_layout(xaxis_title="Motivo", yaxis_title="Frequência")

        fig2 = px.histogram(dff, x="HORA_OCOR", nbins=24, title="Distribuição por Hora")
        fig2.update_layout(xaxis_title="Hora do Dia", yaxis_title="Quantidade")

        fig3 = px.bar(dff["LOCAL_REF"].value_counts().nlargest(10).reset_index(),
                      x="index", y="LOCAL_REF", title="Top 10 Locais")
        fig3.update_layout(xaxis_title="Local", yaxis_title="Quantidade")

        return fig1, fig2, fig3

    app.run_server(mode='inline')


In [ ]:
# ✅ Executar dashboards interativos por classificação
# Use as chamadas abaixo individualmente no Colab para abrir o dashboard desejado:

# iniciar_dashboard_por_classificacao("RECLAMAÇÃO")
# iniciar_dashboard_por_classificacao("ELOGIO")
# iniciar_dashboard_por_classificacao("SUGESTÃO")
